In this notebook we give a proof of concept of unitary compiling using TFQ. 

In [1]:
%load_ext autoreload
%autoreload 2

import sympy 
import numpy as np 
import pandas as pd 
import tensorflow as tf
from utilities.circuit_database import CirqTranslater
from utilities.templates import *
from utilities.variational import Minimizer
from utilities.misc import get_qubits_involved, reindex_symbol, shift_symbols_down
import matplotlib.pyplot as plt 
import tensorflow_quantum as tfq
import cirq
from utilities.compiling import *

In [2]:
def get_positional_dbs(circuit, circuit_db):

    qubits_involved = get_qubits_involved(circuit, circuit_db)
    
    gates_on_qubit = {q:[] for q in qubits_involved}
    on_qubit_order = {q:[] for q in qubits_involved}

    for order_gate, ind_gate in enumerate( circuit_db["ind"]):
        if ind_gate < translator.number_of_cnots:
            control, target = translator.indexed_cnots[str(ind_gate)]
            gates_on_qubit[control].append(ind_gate)
            gates_on_qubit[target].append(ind_gate)
            on_qubit_order[control].append(order_gate)                
            on_qubit_order[target].append(order_gate)  
        else:
            gates_on_qubit[(ind_gate-translator.n_qubits)%translator.n_qubits].append(ind_gate)
            on_qubit_order[(ind_gate-translator.n_qubits)%translator.n_qubits].append(order_gate)        
    return gates_on_qubit, on_qubit_order

In [10]:
translator = CirqTranslater(2)
db1 = u1_layer(translator)
circuit_db = concatenate_dbs([db1]*2)
circuit, circuit_db  = translator.give_circuit(circuit_db)
gates_on_qubit, on_qubit_order = get_positional_dbs(circuit, circuit_db)
simplified_db = circuit_db.copy()

In [31]:
type_get = lambda x: (x-translator.number_of_cnots)//translator.n_qubits


In [38]:
translator = CirqTranslater(6)
indi = translator.number_of_cnots + 9
translator.give_circuit(pd.DataFrame([gate_template(indi)]))

((0, 3): ───Rx(th_0)───,
    ind symbol param_value  trainable  block_id
 0   39   th_0        None       True         0)

In [39]:
type_get(indi)

1

In [254]:
translator = CirqTranslater(3)

u1db = u1_layer(translator)
uflip = pd.DataFrame([gate_template(k) for k in [translator.number_of_cnots, 0,0]])
x_layer_db = pd.DataFrame([gate_template(k, param_value=0.) for k in [translator.number_of_cnots + translator.n_qubits+j for j in range(translator.n_qubits)]])
uflip_x = pd.DataFrame([gate_template(k) for k in [translator.number_of_cnots + 1, translator.number_of_cnots+3, 2, 2]])

circuit_db = concatenate_dbs([x_on_all,uflip_x, uflip_x, uflip_x])
circuit, circuit_db  = translator.give_circuit(circuit_db)
gates_on_qubit, on_qubit_order = get_positional_dbs(circuit, circuit_db)
simplified_db = circuit_db.copy()
circuit

(0, 0): ───Rx(th_0)───Rx(th_4)───X───X───Rx(th_6)───X───X───Rx(th_8)───X───X───
                                 │   │              │   │              │   │
(0, 1): ───Rx(th_1)───Rz(th_3)───@───@───Rz(th_5)───@───@───Rz(th_7)───@───@───

(0, 2): ───Rx(th_2)────────────────────────────────────────────────────────────

In [263]:
type_get = lambda x: (x-translator.number_of_cnots)//translator.n_qubits
check_rot = lambda ind_gate: translator.number_of_cnots<= ind_gate <(3*translator.n_qubits + translator.number_of_cnots)

simplification = False
for q, qubit_gates_path in gates_on_qubit.items():
    if simplification is True:
        break
    for order_gate_on_qubit, ind_gate in enumerate(qubit_gates_path[:-1]):
        if simplification is True:
            break
        ind_gate_p1 = qubit_gates_path[order_gate_on_qubit+1]

        if (check_rot(ind_gate) == True) and (check_rot(ind_gate_p1) == False):
            type_0 = type_get(ind_gate)

            control, target = translator.indexed_cnots[str(ind_gate_p1)]
            
            this_qubit = q
            other_qubits = [control, target]
            other_qubits.remove(q)
            other_qubit = other_qubits[0]
            

            if ((type_0 == 0) and (q==control)) or ((type_0== 1) and (q==target)):
                ### free to pass...
                print("0")
                if len(gates_on_qubit[other_qubit]) == 1:
                    simplification = True
                for qord_other, ind_gate_other in enumerate(gates_on_qubit[other_qubit]):
                    if (ind_gate_other == ind_gate_p1): ## check if we find the same cnot on both qubits
                        print("here")
                        cnot_call__q = on_qubit_order[q][order_gate_on_qubit+1]
                        if cnot_call__q == on_qubit_order[other_qubit][qord_other]:## now check if we are applying the gate on both qubits at same time
                            ### it might happen that there's no gate on the other qbit before the cnot, in that case free to comute.
                            if qord_other == 0:
                                simplification = True
                                break
                            else:
                                gate_in_other_qubit_before_cnot = simplified_db.loc[on_qubit_order[other_qubit][qord_other-1]]["ind"]
                                if check_rot(gate_in_other_qubit_before_cnot) == True:
                                    type_gate_other = type_get(gate_in_other_qubit_before_cnot)
                                    print(q, ind_gate, order_gate_on_qubit)
                                    if type_0 != type_gate_other:
                                        simplification = True
                                        print("okkk")
                                        break
                if simplification == True:

                    info_rot = simplified_db.loc[on_qubit_order[q][order_gate_on_qubit]].copy()
                    info_cnot_control = simplified_db.loc[on_qubit_order[q][order_gate_on_qubit+1]].copy()

                    simplified_db.loc[on_qubit_order[q][order_gate_on_qubit]]  = info_cnot_control
                    simplified_db.loc[on_qubit_order[q][order_gate_on_qubit+1]] = info_rot

                    break

0
here
0 9 1
okkk


In [265]:
circuit

(0, 0): ───Rx(th_0)───Rx(th_4)───X───X───Rx(th_6)───X───X───Rx(th_8)───X───X───
                                 │   │              │   │              │   │
(0, 1): ───Rx(th_1)───Rz(th_3)───@───@───Rz(th_5)───@───@───Rz(th_7)───@───@───

(0, 2): ───Rx(th_2)────────────────────────────────────────────────────────────

In [266]:
scircuit, scircuit_db  = translator.give_circuit(simplified_db)


In [267]:
scircuit

(0, 0): ───Rx(th_0)──────────────X───Rx(th_4)───X───Rx(th_6)───X───X───Rx(th_8)───X───X───
                                 │              │              │   │              │   │
(0, 1): ───Rx(th_1)───Rz(th_3)───@──────────────@───Rz(th_5)───@───@───Rz(th_7)───@───@───

(0, 2): ───Rx(th_2)───────────────────────────────────────────────────────────────────────

In [261]:
circuit_db

,ind,symbol,param_value,trainable,block_id
0,9,th_0,0.0,True,0
1,10,th_1,0.0,True,0
2,11,th_2,0.0,True,0
3,7,th_3,NaN,True,0
4,9,th_4,NaN,True,0
5,2,None,NaN,True,0
6,2,None,NaN,True,0
7,7,th_5,NaN,True,0
8,9,th_6,NaN,True,0
9,2,None,NaN,True,0


In [260]:
simplified_db

,ind,symbol,param_value,trainable,block_id
0,9,th_0,0.0,True,0
1,10,th_1,0.0,True,0
2,11,th_2,0.0,True,0
3,7,th_3,NaN,True,0
4,9,th_4,NaN,True,0
5,2,None,NaN,True,0
6,2,None,NaN,True,0
7,7,th_5,NaN,True,0
8,9,th_6,NaN,True,0
9,2,None,NaN,True,0


In [268]:
def rule_6(translator, simplified_db, on_qubit_order, gates_on_qubit):
    """
    move cnots to the left, rotations to the right.
    
    IMPORTANT this won't work if the cirucit is too short!
    """
    type_get = lambda x: (x-translator.number_of_cnots)//translator.n_qubits
    check_rot = lambda ind_gate: translator.number_of_cnots<= ind_gate <(3*translator.n_qubits + translator.number_of_cnots)

    simplification = False
    for q, qubit_gates_path in gates_on_qubit.items():
        if simplification is True:
            break
        for order_gate_on_qubit, ind_gate in enumerate(qubit_gates_path[:-1]):
            if simplification is True:
                break
            ind_gate_p1 = qubit_gates_path[order_gate_on_qubit+1]

            if (check_rot(ind_gate) == True) and (check_rot(ind_gate_p1) == False):
                type_0 = type_get(ind_gate)

                control, target = translator.indexed_cnots[str(ind_gate_p1)]

                this_qubit = q
                other_qubits = [control, target]
                other_qubits.remove(q)
                other_qubit = other_qubits[0]


                if ((type_0 == 0) and (q==control)) or ((type_0== 1) and (q==target)):
                    ### free to pass...
                    if len(gates_on_qubit[other_qubit]) == 1:
                        simplification = True
                    for qord_other, ind_gate_other in enumerate(gates_on_qubit[other_qubit]):
                        if (ind_gate_other == ind_gate_p1): ## check if we find the same cnot on both qubits
                            cnot_call__q = on_qubit_order[q][order_gate_on_qubit+1]
                            if cnot_call__q == on_qubit_order[other_qubit][qord_other]:## now check if we are applying the gate on both qubits at same time
                                ### it might happen that there's no gate on the other qbit before the cnot, in that case free to comute.
                                if qord_other == 0:
                                    simplification = True
                                    break
                                else:
                                    gate_in_other_qubit_before_cnot = simplified_db.loc[on_qubit_order[other_qubit][qord_other-1]]["ind"]
                                    if check_rot(gate_in_other_qubit_before_cnot) == True:
                                        type_gate_other = type_get(gate_in_other_qubit_before_cnot)
                                        if type_0 != type_gate_other:
                                            simplification = True
                                            break
            if simplification == True:

                info_rot = simplified_db.loc[on_qubit_order[q][order_gate_on_qubit]].copy()
                info_cnot_control = simplified_db.loc[on_qubit_order[q][order_gate_on_qubit+1]].copy()

                simplified_db.loc[on_qubit_order[q][order_gate_on_qubit]]  = info_cnot_control
                simplified_db.loc[on_qubit_order[q][order_gate_on_qubit+1]] = info_rot 
    return simplification, simplified_db

def apply_rule(original_circuit_db, rule, **kwargs):
    max_cnt = kwargs.get('max_cnt',10)
    simplified, cnt = True, 0
    original_circuit, original_circuit_db = translator.give_circuit(original_circuit_db)
    gates_on_qubit, on_qubit_order = get_positional_dbs(original_circuit, original_circuit_db)
    simplified_db = original_circuit_db.copy()
    while simplified and cnt < max_cnt:
        simplified, simplified_circuit_db = rule(translator, simplified_db, on_qubit_order, gates_on_qubit)
        circuit, simplified_db = translator.give_circuit(simplified_circuit_db)
        gates_on_qubit, on_qubit_order = get_positional_dbs(circuit, simplified_db)
        cnt+=1
    return cnt, simplified_db

In [276]:
translator = CirqTranslater(3)

u1db = u1_layer(translator)
uflip = pd.DataFrame([gate_template(k) for k in [translator.number_of_cnots, 0,0]])
x_layer_db = pd.DataFrame([gate_template(k, param_value=0.) for k in [translator.number_of_cnots + translator.n_qubits+j for j in range(translator.n_qubits)]])
uflip_x = pd.DataFrame([gate_template(k) for k in [translator.number_of_cnots + 1, translator.number_of_cnots+3, 2, 2]])

circuit_db = concatenate_dbs([x_on_all,uflip_x, uflip_x, uflip_x])
circuit, circuit_db  = translator.give_circuit(circuit_db)
gates_on_qubit, on_qubit_order = get_positional_dbs(circuit, circuit_db)
simplified_db = circuit_db.copy()

simplification, ssimplified_db = rule_6(translator, simplified_db, on_qubit_order, gates_on_qubit)

An example that you might enter into a loop! It might be good to apply the rules in some order. Like do the commutation only once, then reduce the circuit..

In [277]:
translator.give_circuit(circuit_db)[0]

(0, 0): ───Rx(th_0)───Rx(th_4)───X───X───Rx(th_6)───X───X───Rx(th_8)───X───X───
                                 │   │              │   │              │   │
(0, 1): ───Rx(th_1)───Rz(th_3)───@───@───Rz(th_5)───@───@───Rz(th_7)───@───@───

(0, 2): ───Rx(th_2)────────────────────────────────────────────────────────────

In [278]:
translator.give_circuit(ssimplified_db)[0]

(0, 0): ───Rx(th_0)──────────────X───Rx(th_4)───X───Rx(th_6)───X───X───Rx(th_8)───X───X───
                                 │              │              │   │              │   │
(0, 1): ───Rx(th_1)───Rz(th_3)───@──────────────@───Rz(th_5)───@───@───Rz(th_7)───@───@───

(0, 2): ───Rx(th_2)───────────────────────────────────────────────────────────────────────

In [279]:
simplification, sssimplified_db = rule_6(translator, ssimplified_db, on_qubit_order, gates_on_qubit)

In [280]:
translator.give_circuit(sssimplified_db)[0]

(0, 0): ───Rx(th_0)───Rx(th_4)───X───X───Rx(th_6)───X───X───Rx(th_8)───X───X───
                                 │   │              │   │              │   │
(0, 1): ───Rx(th_1)───Rz(th_3)───@───@───Rz(th_5)───@───@───Rz(th_7)───@───@───

(0, 2): ───Rx(th_2)────────────────────────────────────────────────────────────